# **Project Code**

**By:** Julius Salomons <br>
**Student number:** 14039559 <br>
**Date:** 22 december 2024 <br>
**Course:** Symbolic and Neural AI <br> <br>

**Task description** <br>
This project focuses on a classification task: predicting whether a customer will churn (leave the service) based on their data. The aim is to develop and compare the performance of two models to determine which is better suited for this task. <br>  <br>

**Dataset description** <br>
- https://www.kaggle.com/code/ybifoundation/telecom-customer-churn-prediction  <br>
- 7043 rows and 21 columns, making it manageable for computation on a standard
laptop. <br>
- Fairly balanced dataset (based on churning rates) <br> <br>

**Models**  <br>
1. Nearest Neighbors Classification (NN) <br>
2. Logic tensor network (LTN)  <br>

Model 1 (https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighborsclassification) is a traditional machine learning model. <br>
Model 2 is a neurosymbolic model that uses a neural network to create logic statements for churn prediction. <br>  <br>

**Investigation description** <br>
This investigation will compare model performance using metrics like accuracy, precision, recall, and F1-score. By examining these metrics, we can gain insight into how well each model predicts customer churn. The comparison will provide a deeper understanding of the strengths and limitations of a neural-symbolic approach (LTN) and a traditional machine learning model (NN). <br> <br>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import torch
import shap

In [2]:
# Load dataset
url = "https://raw.githubusercontent.com/dsrscientist/DSData/master/Telecom_customer_churn.csv"
data = pd.read_csv(url)

# displays basic info about the dataset
print("Dataset Head:\n", data.head())

# only 5 features are used in the models
selected_features = ['Dependents', 'tenure', 'MultipleLines', 'MonthlyCharges', 'Contract']

# Handle missing values if any
data = data.dropna()

Dataset Head:
    customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies      

# **Nearest neigbours code**

In [4]:
# encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
  le = LabelEncoder()
  data[column] = le.fit_transform(data[column])
  label_encoders[column] = le

# separate features and target
target_column = 'Churn'
X = data[selected_features]  # Use only the selected 5 features
y = data[target_column]  # Target variable: Churn

# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
# initialize the KNN classifier (choosing 5 neighbors as an example)
knn = KNeighborsClassifier(n_neighbors=5)

# train the model
knn.fit(X_train, y_train)

# make predictions on the test set
y_pred = knn.predict(X_test)

In [6]:
# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print evaluation metrics
print(f"KNN Model Evaluation (using 5 features):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Print classification report
print("\nClassification Report for KNN Model:")
print(classification_report(y_test, y_pred))

KNN Model Evaluation (using 5 features):
Accuracy: 0.7885
Precision: 0.6298
Recall: 0.4879
F1 Score: 0.5498

Classification Report for KNN Model:
              precision    recall  f1-score   support

           0       0.83      0.90      0.86      1036
           1       0.63      0.49      0.55       373

    accuracy                           0.79      1409
   macro avg       0.73      0.69      0.71      1409
weighted avg       0.78      0.79      0.78      1409



# **LTN code**

In [7]:
# encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
  le = LabelEncoder()
  data[column] = le.fit_transform(data[column])

# prepare data
X = data[selected_features]
y = data['Churn']

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# define a neural network model
class ChurnPredictionModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(ChurnPredictionModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 16)  # first hidden layer
        self.fc2 = torch.nn.Linear(16, 1)          # output layer
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return self.sigmoid(x)

In [8]:
# initialize and train the model
input_dim = X_train_scaled.shape[1]
model = ChurnPredictionModel(input_dim)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

# training loop
for epoch in range(num_epochs):
  model.train()
  y_pred = model(X_train_tensor)
  loss = criterion(y_pred, y_train_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  # print progress every 10 epochs
  if (epoch + 1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# after training, evaluate on the test set
model.eval()
with torch.no_grad():
  y_test_pred = model(X_test_tensor)
  y_test_pred = (y_test_pred > 0.5).float()

# calculate accuracy
accuracy = accuracy_score(y_test_tensor.numpy(), y_test_pred.numpy())
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# define SHAP explainer for extracting feature importance
def predict_fn(data):
  data_tensor = torch.tensor(data, dtype=torch.float32)
  with torch.no_grad():
    return model(data_tensor).numpy()

# use a subset for SHAP explanation to speed up (200 samples for background data)
explainer = shap.KernelExplainer(predict_fn, X_train_scaled[:200])
shap_values = explainer.shap_values(X_test_scaled[:200], nsamples=50)

# check SHAP values structure for the selected features
print(f"Shape of shap_values[0]: {shap_values[0].shape}")

# convert SHAP values to a DataFrame for the selected features
shap_df = pd.DataFrame(shap_values[0].T, columns=X.columns)

# define thresholds for features
thresholds = {
  "tenure": 12,
  "MonthlyCharges": 50,
  "Dependents": 0.5,
  "MultipleLines": 0.5,
  "Contract": 1.5
}

# calculate interaction scores for pairs of selected features
interaction_scores = {}
for i, feature1 in enumerate(shap_df.columns):
  for j, feature2 in enumerate(shap_df.columns):
    # avoid duplicates (feature1, feature2) and (feature2, feature1)
    if i < j:
      interaction_scores[(feature1, feature2)] = np.abs(shap_df[feature1] * shap_df[feature2]).sum()

# print the interaction scores for selected feature pairs
print("\nInteraction Scores for Selected Feature Pairs:")
for (feature1, feature2), score in interaction_scores.items():
  print(f"({feature1}, {feature2}): {score}")

# sort the interactions by the highest interaction score
ranked_combinations = sorted(interaction_scores.items(), key=lambda x: x[1], reverse=True)

# print the ranked combinations for selected features
print("\nRanked Feature Combinations for Selected Features (Sorted by Interaction Score):")
for i, ((feature1, feature2), score) in enumerate(ranked_combinations[:10]):  # Display top 10 combinations
  print(f"{i+1}. ({feature1}, {feature2}): {score}")

# generate logical rules from the top combinations based on thresholds
logical_rules = []
for (feature1, feature2), score in ranked_combinations[:10]:
  # create rules based on whether the feature values are higher or lower than the defined thresholds
  rule = f"If ({feature1} {'>' if X[feature1].mean() > thresholds[feature1] else '<'} {thresholds[feature1]}) and ({feature2} {'>' if X[feature2].mean() > thresholds[feature2] else '<'} {thresholds[feature2]}) then Churn is likely"
  logical_rules.append(rule)

# print top logical rules
print("\nTop Logical Rules for Churn Prediction with Specific Thresholds:")
for rule in logical_rules:
  print(rule)


Epoch [10/50], Loss: 0.6393
Epoch [20/50], Loss: 0.6236
Epoch [30/50], Loss: 0.6088


Epoch [40/50], Loss: 0.5948
Epoch [50/50], Loss: 0.5815
Accuracy on test set: 73.53%


  0%|          | 0/200 [00:00<?, ?it/s]

Shape of shap_values[0]: (5, 1)

First few SHAP values:
   Dependents    tenure  MultipleLines  MonthlyCharges  Contract
0    0.014209  0.047986       0.012897       -0.021847  0.009129

Interaction Scores for Selected Feature Pairs:
(Dependents, tenure): 0.0006818356787211146
(Dependents, MultipleLines): 0.00018325723635862615
(Dependents, MonthlyCharges): 0.0003104213688480603
(Dependents, Contract): 0.0001297115473415908
(tenure, MultipleLines): 0.0006188961537168376
(tenure, MonthlyCharges): 0.001048354733646713
(tenure, Contract): 0.0004380617067981406
(MultipleLines, MonthlyCharges): 0.00028176670304482686
(MultipleLines, Contract): 0.000117738012614628
(MonthlyCharges, Contract): 0.00019943766351337663

Ranked Feature Combinations for Selected Features (Sorted by Interaction Score):
1. (tenure, MonthlyCharges): 0.001048354733646713
2. (Dependents, tenure): 0.0006818356787211146
3. (tenure, MultipleLines): 0.0006188961537168376
4. (tenure, Contract): 0.0004380617067981406
5. (Dep

In [9]:
# define a function that applies the comprehensive logical rules based on all features
def rule_based_predictions(data):
  predictions = []

  for _, row in data.iterrows():
    churn_pred = 0  # Default prediction is 0 (no churn)

    # rule 1: MultipleLines is present and MonthlyCharges > 50 (churn likely)
    if row['MultipleLines'] > 0.5 and row['MonthlyCharges'] > 50:
      churn_pred = 1

    # rule 2: MultipleLines is present and Contract is month-to-month (churn likely)
    elif row['MultipleLines'] > 0.5 and row['Contract'] < 1.5:
      churn_pred = 1

    # rule 3: MonthlyCharges > 50 and Contract is month-to-month (churn likely)
    elif row['MonthlyCharges'] > 50 and row['Contract'] < 1.5:
      churn_pred = 1

    # rule 4: Dependents < 0.5 (i.e., no dependents) and MultipleLines is present (churn likely)
    elif row['Dependents'] < 0.5 and row['MultipleLines'] > 0.5:
      churn_pred = 1

    # rule 5: Dependents < 0.5 and MonthlyCharges > 50 (churn likely)
    elif row['Dependents'] < 0.5 and row['MonthlyCharges'] > 50:
      churn_pred = 1

    # rule 6: Dependents < 0.5 and Contract is month-to-month (churn likely)
    elif row['Dependents'] < 0.5 and row['Contract'] < 1.5:
      churn_pred = 1

    # rule 7: Tenure > 12 and MultipleLines > 0.5 (churn likely)
    elif row['tenure'] > 12 and row['MultipleLines'] > 0.5:
      churn_pred = 1

    # rule 8: Tenure > 12 and MonthlyCharges > 50 (churn likely)
    elif row['tenure'] > 12 and row['MonthlyCharges'] > 50:
      churn_pred = 1

    # rule 9: Tenure > 12 and Contract is month-to-month (churn likely)
    elif row['tenure'] > 12 and row['Contract'] < 1.5:
      churn_pred = 1

    # rule 10: Dependents < 0.5 and Tenure > 12 (churn likely)
    elif row['Dependents'] < 0.5 and row['tenure'] > 12:
      churn_pred = 1

    predictions.append(churn_pred)

  return np.array(predictions)

# make rule-based predictions on the test set
rule_based_preds = rule_based_predictions(X_test)

# evaluate rule-based predictions
accuracy = accuracy_score(y_test, rule_based_preds)
precision = precision_score(y_test, rule_based_preds)
recall = recall_score(y_test, rule_based_preds)
f1 = f1_score(y_test, rule_based_preds)

# print evaluation metrics
print(f"Rule-based Model Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print classification report
print("\nClassification Report for Rule-based Model:")
print(classification_report(y_test, rule_based_preds))


Rule-based Model Evaluation:
Accuracy: 0.7168
Precision: 0.4770
Recall: 0.7212
F1 Score: 0.5742

Classification Report for Rule-based Model:
              precision    recall  f1-score   support

           0       0.88      0.72      0.79      1036
           1       0.48      0.72      0.57       373

    accuracy                           0.72      1409
   macro avg       0.68      0.72      0.68      1409
weighted avg       0.77      0.72      0.73      1409

